In [2]:
import os
import random
import numpy as np
import ipywidgets as widgets
from IPython.display import display, Image, clear_output
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from sklearn.neighbors import NearestNeighbors

# Fixer le nombre de cœurs utilisés par joblib (éviter l'avertissement)
os.environ["LOKY_MAX_CPU_COUNT"] = "4"

# Charger le modèle pré-entraîné
model = VGG16(weights='imagenet', include_top=False, pooling='avg')


In [3]:
def extract_features(image_path):
    """ Extrait les caractéristiques d'une image """
    img = load_img(image_path, target_size=(224, 224))
    img_array = img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = preprocess_input(img_array)
    features = model.predict(img_array)
    return features.flatten()


In [4]:
def load_and_extract_features(dataset_dir):
    """ Charge toutes les images et extrait leurs caractéristiques """
    valid_extensions = ('.bmp', '.jpg', '.png')
    image_paths = [os.path.join(dataset_dir, img) for img in os.listdir(dataset_dir) if img.lower().endswith(valid_extensions)]
    feature_list = np.array([extract_features(img) for img in image_paths])
    return image_paths, feature_list

# Charger les données
dataset_dir = "dataset/"
image_paths, feature_list = load_and_extract_features(dataset_dir)

# Initialiser le modèle KNN
knn = NearestNeighbors(n_neighbors=5, metric='euclidean')
knn.fit(feature_list)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 335ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 133ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 119ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 124ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 104ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 101ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 153ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 128ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 120ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 114ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 101ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 122ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 120ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 109ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 142ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 117ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 108ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 98ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 168ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 139ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 119ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 111ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 113ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 149ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 143ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0

NearestNeighbors(metric='euclidean')

In [14]:
# État de l'application
liked_images = []
remaining_images = image_paths.copy()
current_image = random.choice(remaining_images) if remaining_images else None


In [15]:
# Widgets d'affichage
output = widgets.Output()
image_display = widgets.Output()
like_button = widgets.Button(description="Like ❤️", button_style='success')
skip_button = widgets.Button(description="Skip ⏭️", button_style='warning')
recommend_button = widgets.Button(description="Obtenir des recommandations", button_style='info')
liked_images_display = widgets.Output()

def show_image(img_path):
    with image_display:
        clear_output(wait=True)
        display(Image(img_path, width=300))

def update_liked_images():
    with liked_images_display:
        clear_output(wait=True)
        for img in liked_images:
            display(Image(img, width=100))

def like_image(_):
    global current_image
    if current_image:
        liked_images.append(current_image)
        remaining_images.remove(current_image)
        update_liked_images()
        next_image()

def skip_image(_):
    global current_image
    if current_image:
        remaining_images.remove(current_image)
        next_image()

def next_image():
    global current_image
    if remaining_images:
        current_image = random.choice(remaining_images)
        show_image(current_image)
    else:
        with image_display:
            clear_output(wait=True)
            display(widgets.Label("Toutes les images ont été vues !"))

def recommend_images(_):
    with output:
        clear_output(wait=True)
        if not liked_images:
            display(widgets.Label("Veuillez d'abord liker des images pour obtenir des recommandations !"))
        else:
            liked_features = np.array([extract_features(img) for img in liked_images])
            distances, indices = knn.kneighbors(liked_features, n_neighbors=5)
            recommended_images = {image_paths[i] for idx in indices for i in idx}
            for img_path in recommended_images:
                display(Image(img_path, width=200))


In [16]:
# Connecter les boutons aux fonctions
like_button.on_click(like_image)
skip_button.on_click(skip_image)
recommend_button.on_click(recommend_images)

# Affichage initial
display(widgets.Label("Sélectionnez vos image préférées"))
display(image_display)
display(widgets.HBox([like_button, skip_button]))
display(widgets.Label("Images aimées:"))
display(liked_images_display)
display(recommend_button)
display(output)

# Montrer la première image
show_image(current_image)


Label(value='Sélectionnez vos image préférées')

Output()

Label(value='Images aimées:')

Output()

Button(button_style='info', description='Obtenir des recommandations', style=ButtonStyle())

Output()